## Benchmarks Database @Brazil
#### https://www.bcb.gov.br/?SERIESTEMP

In [1]:
from sgs import SGS, series
import pandas as pd
import numpy as np

sgs = SGS()
dtStart = '02/01/1997'
dtEnd ='31/12/2100'

In [2]:
#Lista de Benchmarks
listBMK = {'PTAX': 1,
           'IBOV':7,
           'IRF-M': 12461,
           'IMA-S': 12462,
           'IMA-C': 12463,
           'IMA-B': 12466,
           'IMA-B 5': 12467,
           'IMA-B 5+': 12468,
           'IRF-M 1': 17626,
           'IRF-M 1+': 17627,
          }

li = []

for bmk, cod in listBMK.items():
    frame= sgs.get_valores_series(cod, dtStart, dtEnd)
    if frame[cod].dtype == np.object:
        frame[bmk] = frame[cod].str.replace(',','.').astype(float)
        frame = frame.drop(columns=[cod])
    else:
        frame.columns = [bmk]
    
    li.append(frame)

df1 = pd.concat(li, axis=1)
df1.dropna(axis=0, how='all', inplace=True)

In [3]:
#DF com CDI Numero Indice (acumulado)
df2 = sgs.get_valores_series(4389, dtStart, dtEnd)
df2['CDI'] = df2[4389].str.replace(',','.').astype(float)
df2['FATOR'] = round((1+ df2.CDI / 100) ** (1/252), 8)

df2['CDI_NI'] = df2.FATOR.shift(1).cumprod()
df2 = df2.drop(columns=[4389, 'FATOR'])

In [4]:
#DF Benchmarks
dfBMK = pd.concat([df2.CDI_NI,df1], axis=1)
dfBMK = dfBMK.drop(dfBMK[dfBMK.CDI_NI.isna() & dfBMK.PTAX.isna()].index)
dfBMK.index.name = 'DATA'
dfBMK.tail(10)

,CDI_NI,PTAX,IBOV,IRF-M,IMA-S,IMA-C,IMA-B,IMA-B 5,IMA-B 5+,IRF-M 1,IRF-M 1+
DATA,,,,,,,,,,,
2019-06-14,21.944970,3.8813,98040.0,12815.348161,4532.249234,7153.658515,6988.264333,6067.007079,8580.305977,10827.822029,14267.610198
2019-06-17,21.950373,3.8895,97623.0,12807.724608,4533.368117,7166.406961,6999.878951,6065.121058,8605.074689,10828.775076,14255.715837
2019-06-18,21.955777,3.8608,99404.0,12819.486652,4534.487811,7181.071510,7019.713782,6071.227903,8639.900461,10831.760776,14272.013977
2019-06-19,21.961182,3.8723,100303.0,12833.287843,4535.606056,7196.816577,7031.290667,6077.077620,8658.073009,10833.509741,14291.945991
2019-06-21,21.966589,3.8255,102012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-24,21.971997,3.8234,102062.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-25,21.977407,3.8303,100092.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-26,21.982818,3.8441,100688.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-27,21.988230,3.8642,100723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfBMK.to_csv(r'bmk.csv')